# CaloChallenge dataset reformatting

In [1]:
# imports
from HighLevelFeatures import HighLevelFeatures as HLF
import numpy as np
import h5py
import matplotlib.pyplot as plt

# Dataset 1:

In [2]:
# creating instance of HighLevelFeatures class to handle geometry based on binning file
HLF_1_photons = HLF('photon', filename='binning_dataset_1_photons.xml')
HLF_1_pions = HLF('pion', filename='binning_dataset_1_pions.xml')

In [6]:
# loading the .hdf5 datasets
photon1_file = h5py.File('../dataset_1_photons_1.hdf5', 'r')
photon2_file = h5py.File('../dataset_1_photons_2.hdf5', 'r')
pion_file = h5py.File('../dataset_1_pions_1.hdf5', 'r')

In [8]:
# each file contains one dataset for the incident energy and one for the showers.
print("photon_1:")
for dataset in photon1_file:
    # name of the datasets:
    print("dataset name: ", dataset)
    print("dataset shape:", photon1_file[dataset][:].shape)
print('\n')
print("photon_2:")
for dataset in photon2_file:
    # name of the datasets:
    print("dataset name: ", dataset)
    print("dataset shape:", photon2_file[dataset][:].shape)
print('\n')
print("pion:")
for dataset in pion_file:
    # name of the datasets:
    print("dataset name: ", dataset)
    print("dataset shape:", pion_file[dataset][:].shape)

photon_1:
dataset name:  incident_energies
dataset shape: (121000, 1)
dataset name:  showers
dataset shape: (121000, 368)


photon_2:
dataset name:  incident_energies
dataset shape: (121000, 1)
dataset name:  showers
dataset shape: (121000, 368)


pion:
dataset name:  incident_energies
dataset shape: (120230, 1)
dataset name:  showers
dataset shape: (120230, 533)


<font size="5">__Photon_1:__</font>

In [20]:
# save total number of events in dataset_1_photons_1.hdf5
n_events = photon1_file["incident_energies"].shape[0]
print('Total number of events: ',n_events)

# save number of alpha bins in each layer
num_alpha = HLF_1_photons.num_alpha
print('Number of alpha bins per layer: ', num_alpha)

#save the r edges in each layer
r_edges = HLF_1_photons.r_edges
print('r_edges: ', r_edges)

#save total number of calorimeter layers
n_layers = len(num_alpha)
print('Number of calorimeter layers: ',n_layers)

Total number of events:  121000
Number of alpha bins per layer:  [1, 10, 10, 1, 1]
r_edges:  [[0.0, 5.0, 10.0, 30.0, 50.0, 100.0, 200.0, 400.0, 600.0], [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 15.0, 20.0, 30.0, 40.0, 50.0, 70.0, 90.0, 120.0, 150.0, 200.0], [0.0, 2.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0, 60.0, 80.0, 100.0, 130.0, 160.0, 200.0, 250.0, 300.0, 350.0, 400.0], [0.0, 50.0, 100.0, 200.0, 400.0, 600.0], [0.0, 100.0, 200.0, 400.0, 1000.0, 2000.0]]
Number of calorimeter layers:  5


In [21]:
# save number of r bins per layer
r_bins = []
for layer in range(n_layers):
    r_bins.append(len(r_edges[layer])-1)
print('Number of radius bins per layer: ', r_bins)

Number of radius bins per layer:  [8, 16, 19, 5, 5]


In [33]:
# save layer edges
layer_edges = []
for layer in range(n_layers):
    layer_edges.append(num_alpha[layer]*r_bins[layer])
print('Layer edges: ',layer_edges)

Layer edges:  [8, 160, 190, 5, 5]


368

<font color='blue' size=3>Next, we reformat the dataset to have the structure: 'energies', 'layer_0', 'layer_1',...,'layer_4'. Note that 'energies' has format (n_events,1), while 'layer_i' has format (n_events,num_alpha_i*r_bins_i)</font>

In [42]:
# format 'energies' and store incident energies
energies = photon1_file["incident_energies"]
print('energies shape: ',energies.shape)

#format 'layer_0' and store energies
layer_0 = photon1_file["showers"][:,:layer_edges[0]]
print('layer_0 shape: ',layer_0.shape)

#format 'layer_1' and store energies
layer_1 = photon1_file["showers"][:,(layer_edges[0]):(layer_edges[0]+layer_edges[1])]
print('layer_1 shape: ',layer_1.shape)

#format 'layer_2' and store energies
layer_2 = photon1_file["showers"][:,(layer_edges[0]+layer_edges[1]):(layer_edges[0]+layer_edges[1]+layer_edges[2])]
print('layer_2 shape: ',layer_2.shape)

#format 'layer_3' and store energies
layer_3 = photon1_file["showers"][:,(layer_edges[0]+layer_edges[1]+layer_edges[2]):(layer_edges[0]+layer_edges[1]+layer_edges[2]+layer_edges[3])]
print('layer_3 shape: ',layer_3.shape)

#format 'layer_4' and store energies
layer_4 = photon1_file["showers"][:,(layer_edges[0]+layer_edges[1]+layer_edges[2]+layer_edges[3]):(layer_edges[0]+layer_edges[1]+layer_edges[2]+layer_edges[3]+layer_edges[4])]
print('layer_4 shape: ',layer_4.shape)

energies shape:  (121000, 1)
layer_0 shape:  (121000, 8)
layer_1 shape:  (121000, 160)
layer_2 shape:  (121000, 190)
layer_3 shape:  (121000, 5)
layer_4 shape:  (121000, 5)


In [43]:
# Create reformatted datafile
filename = './reformatted_data/1_gamma.hdf5'
save_file = h5py.File(filename, 'w')

save_file.create_dataset('layer_0', data=layer_0)
save_file.create_dataset('layer_1', data=layer_1)
save_file.create_dataset('layer_2', data=layer_2)
save_file.create_dataset('layer_3', data=layer_3)
save_file.create_dataset('layer_4', data=layer_4)
save_file.create_dataset('energy', data=energies)
    
save_file.close()